In [227]:
# En primer lugar se instalan e importan las librerías y dependencias que serán utilizadas en el proceso.
!pip install SPARQLWrapper  
!pip install wikipedia      
import pandas as pd
from pandas.io.json import json_normalize
from SPARQLWrapper import SPARQLWrapper, JSON, XML
import wikipedia
import json
import xml.etree.ElementTree as ET
import xml.dom.minidom 

In [232]:
print("¿Cual es el identificador de la clase de la cual se quiere generar un diagrama?")
clase = input()

¿Cual es el identificador de la clase de la cual se quiere generar un diagrama?
Q68


In [233]:
#Se define la función para consultar Wikidata. Se consulta el endpoint señalado y 
#devuelve los resultados como un DataFrame de Pandas.

def query_wikidata(sparql_query, sparql_service_url):        
  
    # crear la conexión con el endpoint
    sparql = SPARQLWrapper(sparql_service_url)    
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)

    # solicitar resultados
    result = sparql.query().convert()
    return json_normalize(result["results"]["bindings"])

In [234]:
#Aquí se genera la consulta. La primera es para obtener el numero de instancias de la clase a representar
sparql_query1 = """SELECT DISTINCT ?clase ?claseLabel
WHERE {
?clase wdt:P31 wd:"""+clase+""".
SERVICE wikibase:label { bd:serviceParam wikibase:language "es". }
}
"""
#para consultar otro endpoint, cambiar la URL del servicio y la consulta.
sparql_service_url = "https://query.wikidata.org/sparql"
instancias_wikidata = query_wikidata(sparql_query1, sparql_service_url)

In [235]:
#Se confirma que la consulta haya salido bien
instancias_wikidata.head()

,clase.type,clase.value,claseLabel.type,claseLabel.value,claseLabel.xml:lang
0,uri,http://www.wikidata.org/entity/Q278299,literal,Q278299,NaN
1,uri,http://www.wikidata.org/entity/Q392576,literal,PERM,es
2,uri,http://www.wikidata.org/entity/Q770597,literal,Colossus,es
3,uri,http://www.wikidata.org/entity/Q1151478,literal,Q1151478,NaN
4,uri,http://www.wikidata.org/entity/Q1151495,literal,Q1151495,NaN


In [236]:
numero_de_instancias = instancias_wikidata.shape[0]
numero_de_instancias

144

In [237]:
#Aquí se genera la consulta. Esta se hace para obtener las propiedades de la clase.
sparql_query2 = """SELECT DISTINCT ?p ?pLabel
WHERE {
?clase wdt:P31 wd:"""+clase+""";
?p ?x.
#to obtain property labels:
?property wikibase:directClaim ?p
#to obtain labels in spanish:
SERVICE wikibase:label { bd:serviceParam wikibase:language "es". }
}   
    """
#para consultar otro endpoint, cambiar la URL del servicio y la consulta.
sparql_service_url = "https://query.wikidata.org/sparql"
propiedades_wikidata = query_wikidata(sparql_query2, sparql_service_url)

In [238]:
#Se confirma que la consulta haya salido bien
propiedades_wikidata.head()

,p.type,p.value,pLabel.type,pLabel.value
0,uri,http://www.wikidata.org/prop/direct/P279,literal,http://www.wikidata.org/prop/direct/P279
1,uri,http://www.wikidata.org/prop/direct/P276,literal,http://www.wikidata.org/prop/direct/P276
2,uri,http://www.wikidata.org/prop/direct/P277,literal,http://www.wikidata.org/prop/direct/P277
3,uri,http://www.wikidata.org/prop/direct/P287,literal,http://www.wikidata.org/prop/direct/P287
4,uri,http://www.wikidata.org/prop/direct/P306,literal,http://www.wikidata.org/prop/direct/P306


In [239]:
#Se listan las propiedades de salida de la clase.
lista_completa = propiedades_wikidata["p.value"].tolist()
lista_propiedades = []
for ele in lista_completa:
    ele = ele[36:]
    lista_propiedades.append(ele)
lista_propiedades

['P279',
 'P276',
 'P277',
 'P287',
 'P306',
 'P361',
 'P373',
 'P571',
 'P580',
 'P585',
 'P625',
 'P646',
 'P724',
 'P729',
 'P856',
 'P910',
 'P935',
 'P941',
 'P973',
 'P1296',
 'P1324',
 'P1417',
 'P1448',
 'P1813',
 'P1889',
 'P2013',
 'P2581',
 'P2669',
 'P3222',
 'P3219',
 'P3417',
 'P4342',
 'P5019',
 'P5936',
 'P6366',
 'P7818',
 'P7982',
 'P8349',
 'P9100',
 'P18',
 'P17',
 'P31',
 'P138',
 'P159',
 'P156',
 'P154',
 'P170',
 'P178',
 'P176',
 'P227',
 'P244',
 'P495',
 'P576',
 'P577',
 'P880',
 'P1036',
 'P1366',
 'P1365',
 'P1430',
 'P2043',
 'P2048',
 'P2049',
 'P2671',
 'P2664',
 'P3985',
 'P7772',
 'P127',
 'P136',
 'P144',
 'P155',
 'P162',
 'P186',
 'P195',
 'P217',
 'P527',
 'P582',
 'P1065',
 'P1092',
 'P1482',
 'P1535',
 'P1552',
 'P1705',
 'P1814',
 'P2046',
 'P2283',
 'P2791',
 'P3342',
 'P8408',
 'P61',
 'P131',
 'P366',
 'P641',
 'P1716',
 'P2031',
 'P2130',
 'P2209',
 'P2560',
 'P5204',
 'P7704',
 'P137',
 'P1436',
 'P2149',
 'P2284',
 'P2928',
 'P5868']

In [ ]:
for prop in lista_propiedades:
    #Se obtienen de las propiedades anteriores aquellas que son object properties. Se almacenan en una lista.
    #Aquí se genera la consulta. Esta se hace para obtener las propiedades de la clase.
    sparql= SPARQLWrapper("https://query.wikidata.org/sparql")
    sparql.setQuery("""ASK WHERE {
    wdt:"""+prop+""" rdf:type owl:ObjectProperty
    }   
    """)
    sparql.setReturnFormat(XML)
    results = sparql.query().convert()
    string = results.toxml()
    es_OP = string[105:109]
    if es_OP == "true":
        #Aquí generamos la consulta. Se realiza esta consulta para obtener las clases que pertenecen a dicha propiedad. Se realiza esta
        # consulta para todas las object properties encontradas anteriormente.
        
        from SPARQLWrapper import SPARQLWrapper, JSON

        sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
        sparql.setQuery("""
        SELECT DISTINCT ?claseInicial ?claseInicialLabel ?prop ?propLabel ?Clase ?ClaseLabel
        WHERE {
        #occupation (P106) must be Formula 1 driver (Q10841764)
        ?claseInicial wdt:P31 wd:"""+clase+""" .
        ?claseInicial wdt:"""+prop +""" ?prop .
        ?prop wdt:P31 ?Clase .
        #to obtain labels in spanish:
        SERVICE wikibase:label { bd:serviceParam wikibase:language "es". }
        }  
        """)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
        lista_clases_para_una_propiedad_completo = []
        for result in results["results"]["bindings"]:
            lista_clases_para_una_propiedad_completo.append(result["Clase"]["value"])
        lista_clases_para_una_propiedad = []
        for ele in lista_clases_para_una_propiedad_completo:
            ele = ele[31:]
            lista_clases_para_una_propiedad.append(ele)
        
        # Se obtienen las value type constraint de las proiedades.
        sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
        sparql.setQuery("""SELECT ?Property_ ?Property_Label ?Property_Description ?class_ ?class_Label ?relation_ ?relation_Label
        WHERE {
        ?Property_ wikibase:directClaim wdt:"""+prop+""" .
        ?Property_ p:P2302 ?constraint_statement .
        ?constraint_statement ps:P2302 wd:Q21510865 .
        OPTIONAL {?constraint_statement pq:P2308 ?class_ .}
        OPTIONAL {?constraint_statement pq:P2309 ?relation_ .}

        SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
        } 
        """)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()

        lista_value_type_constraints_completa = []
        for result in results["results"]["bindings"]:
            lista_value_type_constraints_completa.append(result["class_"]["value"])
        lista_value_type_constraints = []
        for ele in lista_value_type_constraints_completa:
            ele = ele[31:]
            lista_value_type_constraints.append(ele)
        
        print("Propiedad:", prop)    
        print("Lista de clases para una propiedad:", lista_clases_para_una_propiedad)
        print("Value Type Constraints:", lista_value_type_constraints)

Propiedad: P279
Lista de clases para una propiedad: []
Value Type Constraints: []
Propiedad: P276
Lista de clases para una propiedad: ['Q3914', 'Q1187811', 'Q1093436', 'Q2398990', 'Q134626', 'Q200250', 'Q1066984', 'Q1180262', 'Q1549591', 'Q38723', 'Q3918', 'Q1637706', 'Q14784328', 'Q17431399', 'Q588140', 'Q515', 'Q22865', 'Q42744322', 'Q85631896', 'Q172754']
Value Type Constraints: ['Q634', 'Q4130', 'Q11446', 'Q22698', 'Q40397', 'Q82794', 'Q188193', 'Q190463', 'Q618123', 'Q712378', 'Q988108', 'Q1299240', 'Q1656682', 'Q2133296', 'Q3895768', 'Q4503801', 'Q4936952', 'Q17334923', 'Q17350442', 'Q20203388', 'Q24334893', 'Q47495022', 'Q89464513']
Propiedad: P277
Lista de clases para una propiedad: ['Q21562092', 'Q9143', 'Q899523', 'Q9143', 'Q7998546', 'Q12772052']
Value Type Constraints: ['Q9143', 'Q165194', 'Q188860']


http://www.wikidata.org/entity/Q5107
http://www.wikidata.org/entity/Q6256
http://www.wikidata.org/entity/Q7275
http://www.wikidata.org/entity/Q161243
http://www.wikidata.org/entity/Q1145276
http://www.wikidata.org/entity/Q3024240
http://www.wikidata.org/entity/Q3502482
http://www.wikidata.org/entity/Q4120211
http://www.wikidata.org/entity/Q15239622


['Q5107',
 'Q6256',
 'Q7275',
 'Q161243',
 'Q1145276',
 'Q3024240',
 'Q3502482',
 'Q4120211',
 'Q15239622']

In [124]:
#Confirmamos que la consulta haya salido bien
clases_object_property_wikidata.head()

,Clase.type,Clase.value,ClaseLabel.type,ClaseLabel.value,ClaseLabel.xml:lang,comic.type,comic.value,comicLabel.type,comicLabel.value,comicLabel.xml:lang,country.type,country.value,countryLabel.type,countryLabel.value,countryLabel.xml:lang
0,uri,http://www.wikidata.org/entity/Q859563,literal,Estado laico,es,uri,http://www.wikidata.org/entity/Q103821919,literal,Q103821919,NaN,uri,http://www.wikidata.org/entity/Q148,literal,República Popular China,es
1,uri,http://www.wikidata.org/entity/Q99541706,literal,Q99541706,NaN,uri,http://www.wikidata.org/entity/Q10366851,literal,Q10366851,NaN,uri,http://www.wikidata.org/entity/Q30,literal,Estados Unidos,es
2,uri,http://www.wikidata.org/entity/Q1520223,literal,república constitucional,es,uri,http://www.wikidata.org/entity/Q103821919,literal,Q103821919,NaN,uri,http://www.wikidata.org/entity/Q148,literal,República Popular China,es
3,uri,http://www.wikidata.org/entity/Q6256,literal,país,es,uri,http://www.wikidata.org/entity/Q11796260,literal,Q11796260,NaN,uri,http://www.wikidata.org/entity/Q36,literal,Polonia,es
4,uri,http://www.wikidata.org/entity/Q842112,literal,Estado socialista,es,uri,http://www.wikidata.org/entity/Q103821919,literal,Q103821919,NaN,uri,http://www.wikidata.org/entity/Q148,literal,República Popular China,es


In [125]:
#Aqui se obtienen el numero de apariciones de cada clase para hacer un recuento y obtener la clase a representar
clases_object_property_wikidata["country.value"].value_counts()

http://www.wikidata.org/entity/Q30       7
http://www.wikidata.org/entity/Q148      6
http://www.wikidata.org/entity/Q36       6
http://www.wikidata.org/entity/Q213      2
http://www.wikidata.org/entity/Q15180    2
Name: country.value, dtype: int64

In [ ]:
# Se van comparando cada clase de mayor numero de apariciones a menor numero de apariciones con las clases del value type 
# constraint de la pripiedad de la que se ha realizado la consulta. De esta manera obtendremos aquella clase a representar 
# en el diagrama
